# Imports
- All library imports
- Original DataFrame import

In [33]:
import pandas as pd
import numpy as np
import re
from collections import Counter
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

shark_df = pd.read_excel('../shark-dataset.xls')
# create dataframe copy
original_df = shark_df.copy()
shark_df

,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,...,Species,Source,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22
0,2024-10-14 00:00:00,2024.0,Unprovoked,Honduras,Tela,Atlantida,Swimming,Roberto Carlose Bu Mejia,M,38,...,Bull shark,Georgienne Bradley: Daily Mail UK: FTN News.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-10-11 00:00:00,2024.0,Unprovoked,USA,Florida,Brevard County Orlando,Surfing,Teddy Witteman,M,16,...,Bull shark 6ft,Todd SmithFlorida today: News 4:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-09-17 00:00:00,2024.0,Unprovoked,Honduras,Trujillo Colon,Puerto Castillo,Diving for shellfish,Fernando Mendoza Ocampo,M,33,...,Bull shark 6-7ft,Daily Mail UK: The Sun UK: The World Watch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-09-16 00:00:00,2024.0,Unprovoked,Morocco,Southern Morocco,West of Dakhla,Swimming - jumped off yacht,German Tourist,F,30,...,Reportedly a Great White,Andy Currie: Moroccan World News: Sky News,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-08-26 00:00:00,2024.0,Unprovoked,Jamaica,Montego Bay,Falmouth,Spearfishing,Jahmari Reid,M,16,...,Reportedly Tiger Shark,Todd Smith: Daily Mail UK: Sky News: People .com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6968,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,...,NaN,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005,6.0,NaN,NaN
6969,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,...,NaN,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004,5.0,NaN,NaN
6970,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,...,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003,4.0,NaN,NaN
6971,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,...,NaN,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,ND.0002,3.0,NaN,NaN


### Column cleaning

In [35]:
unused_columns = ['type', 'state', 'name', 'location', 'species', 'source', 'pdf', 'href_formula', 'href', 'case_number', 'case_number.1', 'original_order', 'unnamed:_21', 'unnamed:_22', 'time', 'injury']

def clean_columns(df):
    df.columns = df.columns.str.lower().str.strip().str.replace(" ", "_", regex=False) # lowercase col names, remove+replace empty spaces
    df.rename(columns={'unnamed:_11': 'fatal'}, inplace=True)
    df = df.drop(unused_columns, axis=1, errors='ignore')
    return df

shark_df = clean_columns(shark_df)
shark_df.head()

,date,year,country,activity,sex,age,fatal
0,2024-10-14 00:00:00,2024.0,Honduras,Swimming,M,38,N
1,2024-10-11 00:00:00,2024.0,USA,Surfing,M,16,N
2,2024-09-17 00:00:00,2024.0,Honduras,Diving for shellfish,M,33,N
3,2024-09-16 00:00:00,2024.0,Morocco,Swimming - jumped off yacht,F,30,Y
4,2024-08-26 00:00:00,2024.0,Jamaica,Spearfishing,M,16,Y


### Year filtering

In [37]:
start_year = 2014
end_year = 2024

def filter_years(df):
    df = df[(df['year'] >= start_year) & (shark_df['year'] <= end_year)]
    df['year'] = df['year'].fillna(0).astype(int) #convert float in year to int
    return df

shark_df = clean_columns(shark_df)
shark_df.head() 

,date,year,country,activity,sex,age,fatal
0,2024-10-14 00:00:00,2024.0,Honduras,Swimming,M,38,N
1,2024-10-11 00:00:00,2024.0,USA,Surfing,M,16,N
2,2024-09-17 00:00:00,2024.0,Honduras,Diving for shellfish,M,33,N
3,2024-09-16 00:00:00,2024.0,Morocco,Swimming - jumped off yacht,F,30,Y
4,2024-08-26 00:00:00,2024.0,Jamaica,Spearfishing,M,16,Y


## Years (Tung)

In [39]:
# reduce range rows for better valid inputs

start_year = 2014
end_year = 2024

shark_df = shark_df[(shark_df["year"] >= start_year) & (shark_df["year"] <= end_year)]

#convert float in year to int
shark_df["year"] = shark_df["year"].fillna(0).astype(int)

# remove invalid rows with "2014" as date
shark_df = shark_df.drop(shark_df.index[-2:])


total_count_spring = 298
total_count_summer = 405
total_count_autumn = 290
totao_count_winter = 231

shark_df

,date,year,country,activity,sex,age,fatal
0,2024-10-14 00:00:00,2024,Honduras,Swimming,M,38,N
1,2024-10-11 00:00:00,2024,USA,Surfing,M,16,N
2,2024-09-17 00:00:00,2024,Honduras,Diving for shellfish,M,33,N
3,2024-09-16 00:00:00,2024,Morocco,Swimming - jumped off yacht,F,30,Y
4,2024-08-26 00:00:00,2024,Jamaica,Spearfishing,M,16,Y
...,...,...,...,...,...,...,...
1226,07-Feb-2014,2014,TRINIDAD & TOBAGO,Fishing,M,NaN,N
1227,07-Feb-2014,2014,NEW ZEALAND,Surfing,M,28,N
1228,26-Jan-2014,2014,AUSTRALIA,Fishing,M,NaN,N
1229,25-Jan-2014,2014,NEW ZEALAND,Spearfishing,M,24,N


## Date & Time (Tung)

In [ ]:
# Function to parse different date formats
def parse_date(date_str):
    if isinstance(date_str, str):
        try: 
            return pd.to_datetime(date_str)  # Try direct conversion
        except ValueError:
            match = re.search(r'(\d{4}-\d{1,2}-\d{1,2}|\d{1,2}-[A-Za-z]{3}-\d{4}|\b[A-Za-z]{3}-\d{4}\b)', date_str)
            if match:
                date_str = match.group(0)
                try:
                    return datetime.strptime(date_str, "%Y-%m-%d")
                except ValueError:
                    try:
                        return datetime.strptime(date_str, "%d-%b-%Y")
                    except ValueError:
                        try:
                            return datetime.strptime(date_str, "%b-%Y")
                        except ValueError:
                            return None  # Return None for invalid formats
    elif isinstance(date_str, datetime):
        return date_str  # Return the datetime object as is
    return None  # Return None if not a string or datetime

# Create datetime_column and string_column
dft["datetime_column"] = dft["date"].apply(parse_date)
dft["string_column"] = dft["date"].apply(lambda x: x if isinstance(x, str) else None)

# Drop rows with invalid datetime values
dft = dft[dft["datetime_column"].notna()]

# Extract month and year from datetime_column
dft['month'] = dft["datetime_column"].apply(lambda x: x.month if pd.notnull(x) else None)
dft['year'] = dft["datetime_column"].apply(lambda x: x.year if pd.notnull(x) else None)
#dft['month'] = dft["datetime_column"].dt.month
#dft['year'] = dft["datetime_column"].dt.year

# Define season mapping
season_mapping = {
    "Spring": [3, 4, 5],
    "Summer": [6, 7, 8],
    "Autumn": [9, 10, 11],
    "Winter": [12, 1, 2]
}

# Function to assign season based on month
def what_season(month):
    for season, months in season_mapping.items():
        if month in months:
            return season
    return None

# Assign season based on the extracted month
dft['season'] = dft['month'].apply(what_season)

# Check the resulting DataFrame
print(dft[['date', 'datetime_column', 'string_column', 'year', 'month', 'season']])

## Fatality rates (Bru)

In [ ]:
# fatality wrangling with tung's dataframe (2014 to 2024)
bru_df = shark_df.copy()

# copy of tung's df
fatal_df = dft.copy()

# rename column
fatal_df.rename(columns={'unnamed:_11': 'fatal'}, inplace=True)

replacement_dict = {
    'N': 'no',
    'Y': 'yes',
    'M': 'unknown',
    'F': 'unknown',
    'n': 'no',
    'Nq': 'unknown'
}

#fill NaN vals with 'unknown' and replace unique values
fatal_df['fatal'] = fatal_df['fatal'].fillna('unknown').replace(replacement_dict)

fatality_counts = fatal_df['fatal'].value_counts()
display("fatality counts", fatality_counts)

## Activity (Bru)

In [ ]:
# Bru - activity wrangling

import re

# check unique values
unique_values_activity = bru_df['activity'].unique()
# print(unique_values_activity)

# convert all values to a common case
bru_df['activity'] = bru_df['activity'].str.strip().str.lower().str.replace(r"[\"']", '', regex=True)

'''
The word_count() function uses collections and re to concatenate all the values in the column to a single string.
Then it will split the combined string into individual words.
Using Counters, it will return the frequency of each word.
It will also store the top 10 most common words in a list
'''

most_common_words = []

def word_count():
    bru_df['activity'] = bru_df['activity'].fillna('').astype(str) # replace NaN values with an empty string and convert all to string
    all_text = ' '.join(bru_df['activity']) # combine all values into a single string
    words = re.findall(r'\w+', all_text.lower()) # split into words (regex to handle punctuation)
    word_counts = Counter(words) # count word frequency
    most_common_words = [word for word, count in word_counts.most_common(50) if len(word) >= 5]
    return most_common_words
    return word_count

word_count()
most_common_words = word_count()

print('Top values before replacement function\n', bru_df['activity'].value_counts().head(10))
#manually entered selected values
selected_values_to_replace = ['surfing', 'diving', 'fishing', 'swimming', 'wading', 'bathing', 'snorkeling', 'kayaking', 'body boarding', 'scuba diving']


def replace_values():
    for word_to_replace in selected_values_to_replace:
        bru_df.loc[bru_df['activity'].str.contains(word_to_replace, case=False, na=False), 'activity'] = word_to_replace

    return bru_df

updated_bru_df = replace_values()

print('\nTop values after replacement:\n', updated_bru_df['activity'].value_counts().head(10))

# print("Word count:")
# print(word_counts_df)

bru_df.head()

In [ ]:
# fatality wrangling with tung's dataframe (2014 to 2024)
# bru_df = shark_df.copy()

# # copy of tung's df
# fatal_df = dft.copy()

# rename column
bru_df.rename(columns={'unnamed:_11': 'fatal'}, inplace=True)

replacement_dict = {
    'N': 'no',
    'Y': 'yes',
    'M': 'unknown',
    'F': 'unknown',
    'n': 'no',
    'Nq': 'unknown'
}

#fill NaN vals with 'unknown' and replace unique values
bru_df['fatal'] = bru_df['fatal'].fillna('unknown').replace(replacement_dict)

fatality_counts = bru_df['fatal'].value_counts()
display("fatality counts", fatality_counts)

bru_df.head()

In [ ]:
shark_final_df = bru_df.to_csv('shark_final_df.csv', index=False)
shark_final_df